# Implicit feedback recommender systems for Z+
> study carried out based on the data for DE shop during 2017

A common task of recommender systems is to improve customer experience through personalized recommendations based on prior implicit feedback. These systems passively track different sorts of user behavior, such as purchase history, watching habits and browsing activity, in order to model user preferences. Unlike the much more extensively researched explicit feedback, in case of implicit feedback we do not have any direct input from the users regarding their preferences. In particular, we lack substantial evidence on which products consumer dislike. 

Recommender systems rely on different types of input.Most convenient is the high quality explicit feedback, which includes explicit input by users regarding their interest in products. For example, Netflix collects star ratings for movies and TiVo users indicate their preferences for TV shows by hitting thumbs-up/down buttons. However, explicit feedback is not always vailable. Thus, recommenders can infer user preferences from the more abundant implicit feedback, which indirectly reflect opinion through observing user behavior [ref here]. Types of implicit feedback include purchase history, browsing history, search patterns, or even mouse movements. 

As per the famous [work Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf):
It is crucial to identify the unique characteristics of implicit feedback, which prevent the direct use of algorithms that were designed with explicit feedback in mind. 
1. No negative feedback. By observing the users behavior, we can infer which items they probably like and thus chose to consume. However, it is hard to reliably infer which items a user did not like. For example, a user that did not watch a certain show might have done so because she dislikes the show or just because she did not know about the show or was not available to watch it. This fundamental asymmetry does not exist in explicit feedback where users tell us both what they like and what they dislike. 
2. Implicit feedback is inherently noisy. While we passively track the users behavior, we can only guess their preferences and true motives. 
3. The numerical value of explicit feedback indicates preference, whereas the numerical value of implicit feedback indicates confidence.

## Data loading and preaparation

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from IPython.display import display, HTML

For the sake of tiem and computational resources we will be using an sample of data avalilable from the Z+ DE shop, restricted only to Cats and Dogs categories
For the specific SQL statements used for construction of the export file please refer to [sql_export.sql](linkhere)

In [3]:
raw_df = pd.read_csv('export_with_basket_2017.csv', encoding = "utf-8")#, nrows = 100000)

In [4]:
raw_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID,SCM_N_ORDER_NR_1
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3,80391417
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2,81849313
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2,81849313
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3,83341895


In [143]:
#raw_df.info()

In [6]:
## should we have any missing customer data
# fortunately this is not the case for us, but I will leave the cleaning step as must do one anyway
cleaned_df = raw_df.loc[pd.isnull(raw_df.CUS_N_KEY_CUSTOMER) == False]

In [144]:
#cleaned_retail.info()

User-Items intereaction matrix will be a sparse matrix with ideces for items and users instead of real IDs. So before we make any sort of ratings matrix, we have create a lookup table that keeps track of each item ID along with a description of that item. We will do that in straightforward manner, simoply creating corresponding dictionaries

In [8]:
item_lookup = cleaned_df[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']].drop_duplicates() # Only get unique item/description pairs
item_lookup['ART_C_PROD_NUMBER'] = item_lookup.ART_C_PROD_NUMBER.astype(int) # Encode as strings for future lookup ease
item_lookup['ART_V_ART_DESCRIPTION'] = item_lookup.ART_V_ART_DESCRIPTION.astype(str) # Encode as strings for future lookup ease

In [9]:
item_lookup.head()

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,139428,139428.16 Royal Canin British Shorthair Adult ...
1,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis"
2,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...
3,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...
5,451643,451643.2 Felix Häppchen in Gelee oder Leckerbi...


This can tell us what each item is, such as that ART_C_PROD_NUMBER 3481 is a  Dog Design Decke Hundeschar . Now that this has been created, we need to:

1. Group purchase quantities together by item prod number and item ID
2. Change any sums that equal zero to one (we can actually ommit this situation, since as far as I can tell we do not store "returned" type information)
3. Only include customers with a positive purchase total to eliminate possible errors
4. *Set up our sparse ratings matrix*

This last step is especially important if you don’t want to have unnecessary memory issues! If you think about it, our matrix is going to contain thousands of items and thousands of users with a user/item value required for every possible combination. That is a LARGE matrix, so we can save a lot of memory by keeping the matrix sparse and only saving the locations and values of items that are not zero.

The code below will finish the preprocessing steps necessary for our final ratings sparse matrix:

In [10]:
cleaned_df['CUS_N_KEY_CUSTOMER '] = cleaned_df.CUS_N_KEY_CUSTOMER.astype(int) # Convert to int for customer ID
cleaned_df = cleaned_df[['ART_C_PROD_NUMBER', 'SCM_D_QUANTITY', 'CUS_N_KEY_CUSTOMER']] # Get rid of unnecessary info
grouped_cleaned = cleaned_df.groupby(['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']).sum().reset_index() # Group together
grouped_cleaned.SCM_D_QUANTITY.loc[grouped_cleaned.SCM_D_QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_cleaned.query('SCM_D_QUANTITY > 2') # Only get customers where purchase totals were positive

## If we look at our final resulting matrix of grouped purchases, we see the following:
grouped_purchased.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,CUS_N_KEY_CUSTOMER,ART_C_PROD_NUMBER,SCM_D_QUANTITY
0,22,52735,5.0
7,22,397041,4.0
8,22,399251,3.0
12,22,447409,4.0
18,22,543556,3.0


# ToDo: 
Actually here I, perhaps, do not have to filter based on the SCM_D_QUANTITY, but I have to filter the items that have less than 5 interactions, at the same time we have to remove the users that ineracted with less than 5 different items. Check for the remaining items agan, and do it untill the DF is stabilized.

In the next two cells we will do exactly this and this would lead to a substantial reduction in the sparcity of the item-user interaction matrix, making it suitable for matrix factorization procedure

In [11]:
def lower_limit(df, CUS_N_KEY_CUSTOMER_min, ART_C_PROD_NUMBER_min):
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of items: {}'.format(n_items))
    print('Weak-Sparsity: {:4.3f}%'.format(sparsity)) # this is not exactly a sparsity. I have to change it a math correct one
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        ART_C_PROD_NUMBER_counts = df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count()
        df = df[~df.CUS_N_KEY_CUSTOMER.isin(ART_C_PROD_NUMBER_counts[ART_C_PROD_NUMBER_counts < ART_C_PROD_NUMBER_min].index.tolist())]
        CUS_N_KEY_CUSTOMER_counts = df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count()
        df = df[~df.ART_C_PROD_NUMBER.isin(CUS_N_KEY_CUSTOMER_counts[CUS_N_KEY_CUSTOMER_counts < CUS_N_KEY_CUSTOMER_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('CUS_N_KEY_CUSTOMER').ART_C_PROD_NUMBER.count().min() >= ART_C_PROD_NUMBER_min)
    assert(df.groupby('ART_C_PROD_NUMBER').CUS_N_KEY_CUSTOMER.count().min() >= CUS_N_KEY_CUSTOMER_min)
    
    n_users = df.CUS_N_KEY_CUSTOMER.unique().shape[0]
    n_items = df.ART_C_PROD_NUMBER.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of items: {}'.format(n_items))
    print('Weak-Sparsity: {:4.3f}%'.format(sparsity)) # this is not exactly a sparsity. I have to change it a math correct one
    return df

In [12]:
df_lim = lower_limit(grouped_purchased, 5, 5)

Starting likes info
Number of users: 594238
Number of models: 6869
Sparsity: 0.051%
Ending likes info
Number of users: 137953
Number of models: 5564
Sparsity: 0.163%


In [13]:
df_lim.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1249663 entries, 0 to 11278530
Data columns (total 3 columns):
CUS_N_KEY_CUSTOMER    1249663 non-null int64
ART_C_PROD_NUMBER     1249663 non-null int64
SCM_D_QUANTITY        1249663 non-null float64
dtypes: float64(1), int64(2)
memory usage: 38.1 MB


In [14]:
customers = list(np.sort(df_lim.CUS_N_KEY_CUSTOMER.unique())) # Get our unique customers
products = list(df_lim.ART_C_PROD_NUMBER.unique()) # Get our unique products that were purchased
quantity = list(df_lim.SCM_D_QUANTITY) # All of our purchases

rows = df_lim.CUS_N_KEY_CUSTOMER.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = df_lim.ART_C_PROD_NUMBER.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [147]:
# sanity check
purchases_sparse

<137953x5564 sparse matrix of type '<class 'numpy.float64'>'
	with 1249663 stored elements in Compressed Sparse Row format>

In [16]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.83719241255042

# Creating a Training and Validation Set

As per above cited [Collaborative Filtering for Implicit Feedback Dataset](http://yifanhu.net/PUB/cf.pdf), one of the main issues with implicit factorization modes is _Evaluation_
**"Evaluation of implicit-feedback recommender requires appropriate measures. In the traditional setting where a user is specifying a numeric score, there are clear metrics
such as mean squared error to measure success in prediction. However with implicit models we have to take into account availability of the item, competition for the item with other items, and repeat feedback. For example, if we gather data on television viewing, it is unclear how to evaluate a show that has been watched more than once, or how to compare two shows that are
on at the same time, and hence cannot both be watched by the user."**

Usually split in train and test set is fairly simple: just take a random sample of the training example rows in our feature matrix and separate it away from the training set. That normally looks like this:

With collaborative filtering, that’s not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end. Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.

Our test set is an exact copy of our original data. The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero). We then check in the test set which items were recommended to the user that they ended up actually purchasing. If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.

As an additional check, we can compare our system to simply recommending the most popular items to every user (beating popularity is a bit difficult). This will be our baseline.

This method of testing isn’t necessarily the “correct” answer, because it depends on how you want to use the recommender system. However, it is a practical way of testing performance I will use for this example.

In [17]:
import random

In [18]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

In [19]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

# SLOW Python implementation

### Imlementing ALS for Implicit feedback case

Now that we have our training and test sets finished, we can move on to implementing the algorithm. If we look at the paper of [Hu, Koren, and Volinsky](http://yifanhu.net/PUB/cf.pdf)
we can see that the key equations will we need to implement into the algorithm. First, we have our ratings matrix which is sparse (represented by the product_train sparse matrix object). We need to turn this into a confidence matrix (from page 4):
$$
C_{ui}=1+\alpha r_{ui}
$$

Where $C_{ui}$ is the confidence matrix for our users $u$ and our items $i$. The $\alpha$ term represents a linear scaling of the rating preferences (in our case number of purchases) and the rui term is our original matrix of purchases. The paper suggests 40 (forty, Karl!) as a good starting point.

After taking the derivative of equation 3 in the paper, we can minimize the cost function for our users U:

$$
x_u=\left(Y^TC^uY+\lambda I\right)^{−1}Y^TC^u p(u)
$$

The authors note you can speed up this computation through some linear algebra that changes this equation to:
$$
x_u=\left(Y^TY+Y^T(C^u−I)Y+\lambda I\right)^{−1}Y^TC^u p(u)
$$

Notice that we can now precompute the YTY portion without having to iterate through each user u. We can derive a similar equation for our items:
$$
y_i=(X^TX+X^T(C^i−I)X+\lambda I)^{−1}X^TC^i p(i)
$$
These will be the two equations we will iterate back and forth between until they converge. We also have a regularization term λ that can help prevent overfitting during the training stage as well, along with our binarized preference matrix p which is just 1 where there was a purchase (or interaction) and zero where there was not.

!!! Shoutout to Chris Johnson’s implicit-mf code that was a helpful guide for this. The code blow is altered versino of his to make things easier to understand.

In [20]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

### That is slow, oh, that is slow! Do not RUN

In [212]:
# user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
#                                             rank_size = 20)

We can investigate ratings for a particular user by taking the dot product between the user and item vectors (U and V). Let’s look at our first user.

In [148]:
# user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

In [ ]:
# array([ 0.00644811, -0.0014369 ,  0.00494281,  0.00027502,  0.01275582])

# Speeding Up ALS

This code in its raw form is just too slow. We have to do a lot of looping, and we haven’t taken advantage of the fact that our algorithm is embarrasingly parallel, since we could do each iteration of the item and user vectors independently. Fortunately, as I was still finishing this up, Ben Frederickson at Flipboard had perfect timing and came out with a version of ALS for Python utilizing Cython and parallelizing the code among threads. One can read his blog post about using it for finding similar music artists using matrix factorization here and his implicit library here. He claims it is even faster than Quora’s C++ QMF, but I haven’t tried theirs. The implementation it is over 1000 times faster than this bare bones pure Python version 

First, import his library so we can utilize it for our matrix factorization.

In [25]:
import implicit

In [63]:
type(product_train*alpha)

scipy.sparse.csr.csr_matrix

In [126]:
alpha = 15

# changing deprecated 
# user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
#                                                           factors=20, 
#                                                           regularization = 0.1, 
#                                                          iterations = 50)
model = implicit.als.AlternatingLeastSquares(factors=10, 
                                             regularization = 0.1, 
                                             iterations=10, 
                                             calculate_training_loss=True)
model.fit(((product_train*alpha).T).astype('double'))
user_items = (product_train*alpha).tocsr()


100%|██████████| 10.0/10 [00:23<00:00,  2.01s/it, loss=0.0387]


In [187]:
user_id = 1990
## 123 mixed user
## 512 mixed user
## 712 cat specific
customer_id = customers_arr[user_id]
# print(customer_id)
recommendations = model.recommend(user_id, user_items)
# recommendations
fast_recommended = []
for index in recommendations:
    code = products_arr[index[0]]
    fast_recommended.append(code)
print(fast_recommended)

[480811, 10346, 298490, 276086, 19401, 574037, 157079, 13912, 230350, 298495]


# Evaluating the Recommender System (I have switched to multithreaded Implict libarary, so this can be skipped for now)

In [27]:
from sklearn import metrics

In [28]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

In [29]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

We can now use this function to see how our recommender system is doing. To use this function, we will need to transform our output from the ALS function to csr_matrix format and transpose the item vectors. The original pure Python version output the user and item vectors into the correct format already.

In [30]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.906, 0.869)

We can see that our recommender system beat popularity. Our system had a mean AUC of 0.87, while the popular item benchmark had a lower AUC of 0.869. You can go back and tune the hyperparameters if you wish to see if you can get a higher AUC score. Ideally, you would have separate train, cross-validation, and test sets so that you aren’t overfitting while tuning the hyperparameters, but this setup is adequate to demonstrate how to check that the system is working.

# A Recommendation Example

In [31]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

Now, we can create a function that will return a list of the item descriptions from our earlier created item lookup table.

In [32]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    return item_lookup.loc[item_lookup.ART_C_PROD_NUMBER.isin(prod_codes)]

We need to look these up by a customer’s ID. Looking at the list of customers:

In [33]:
customers_arr[10:105]

array([  610,   830,   883,   921,  1312,  1375,  1442,  1525,  1553,
        1611,  1682,  1775,  1814,  1836,  1921,  2016,  2081,  2236,
        2326,  2372,  2625,  2676,  2974,  3133,  3142,  3299,  3338,
        3445,  3586,  3605,  3644,  3779,  3797,  3826,  4012,  4340,
        4637,  4661,  4891,  4952,  5036,  5211,  5463,  5493,  5755,
        5855,  5909,  5987,  6096,  6174,  6481,  6489,  6675,  6835,
        6854,  6856,  6860,  6912,  7016,  7043,  7110,  7134,  7205,
        7246,  7313,  7342,  7352,  7373,  7432,  7653,  7930,  8029,
        8030,  8164,  8196,  8478,  8552,  8588,  8626,  8642,  8736,
        8752,  8767,  8810,  8829,  8844,  9093,  9229,  9431,  9758,
        9952, 10088, 10450, 10501, 10823])

we can see that the first customer listed has an ID of 12346. Let’s examine their purchases from the training set.

In [34]:
get_items_purchased(22, product_train, customers_arr, products_arr, item_lookup)

[ 52735 397041 447409 543556 616867]


,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
326,397041,"397041.3 Felix ""So gut wie es aussieht"" Pouche..."
536,447409,447409.2 Vitakraft Cat Stick Mini - Lachs & Fo...
540,447409,447409.0 Vitakraft Cat Stick Mini - Ente & Kan...
1245,447409,447409.14 Vitakraft Cat Stick Mini - Truthahn ...
1246,447409,447409.15 Vitakraft Cat Stick Mini - Geflügel ...
1541,447409,447409.13 Vitakraft Cat Stick Mini - Lachs & F...
1620,52735,52735.12 Miamor Feine Filets 6 x 100 g - Thunf...
1652,447409,447409.10 Vitakraft Cat Stick Mini - Truthahn ...
1653,447409,447409.8 Vitakraft Cat Stick Mini - Lachs & Fo...
1655,447409,447409.11 Vitakraft Cat Stick Mini - Geflügel ...


In [35]:
from sklearn.preprocessing import MinMaxScaler

In [36]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        # print(code)
        rec_list.append([code, item_lookup.ART_V_ART_DESCRIPTION.loc[item_lookup.ART_C_PROD_NUMBER == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    # print (descriptions)
    final_frame = pd.DataFrame({'ART_C_PROD_NUMBER': codes, 'ART_V_ART_DESCRIPTION': descriptions}) # Create a dataframe 
    return final_frame[['ART_C_PROD_NUMBER', 'ART_V_ART_DESCRIPTION']], codes # Switch order of columns around
    #return 0

In [37]:
r_df, _ = rec_items(208, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)
r_df

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,508482,508482.1 Royal Canin Veterinary Diet Feline Re...
1,531249,531249.2 Kattovit Niere/Renal Pouch 6 x 85 g -...
2,471676,471676.0 Royal Canin Veterinary Diet Feline Re...
3,479505,479505.2 Kattovit Gastro Pouch 6 x 85 g - 6 x ...
4,508454,508454.2 Royal Canin Veterinary Diet Feline Se...
5,206642,206642.4 Hill's Prescription Diet k/d Kidney C...
6,556340,556340.10 Sparpaket Kattovit Pouches 24 x 85 g...
7,508456,508456.0 Royal Canin Veterinary Diet Feline Ur...
8,529296,529296.1 Kattovit Sensitive 6 x 175 g - 12 x 1...
9,10350,10350.16 Animonda Carny Adult 6 x 400 g - Rind...


These recommendations seem quite good! Remember that the recommendation system has no real understanding of what a kat food is. All it knows is the purchase history. It identified that people purchasing vet related cat food may also want to purchase sensitive cat food. 

Let us select some other user

In [213]:
raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == 221].head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID,SCM_N_ORDER_NR_1
1025086,345861938,81276553,0,23-JAN-17,1500488,74444244,221,1.0,64620,46372,...,447409.11 Vitakraft Cat Stick Mini - Geflügel ...,447409.11 Vitakraft Cat Stick Mini - Geflügel ...,Katzensnacks,Vitakraft,Vitakraft,Vitakraft,Vitakraft,snacks,3,81276553
1671583,345861932,81276553,0,23-JAN-17,1500466,74444252,221,1.0,64616,46372,...,447409.6 Vitakraft Cat Stick Mini - Ente & Kan...,447409.6 Vitakraft Cat Stick Mini - Ente & Kan...,Katzensnacks,Vitakraft,Vitakraft,Vitakraft,Vitakraft,snacks,3,81276553
1671899,345861928,81276553,0,23-JAN-17,1933976,107664472,221,1.0,69844,49693,...,565453.1 My Star is a Foodie - Gemischtes Pake...,565453.1 My Star is a Foodie - Gemischtes Pake...,Katzenfutter nass,★ My Star,My Star zarte Mousse,My Star zarte Mousse,MATINA,Canned Food Cat / Dog,3,81276553
1671900,345861929,81276553,0,23-JAN-17,1888821,55758654,221,1.0,58963,43981,...,52735.10 Miamor Feine Filets 6 x 100 g - Huhn ...,52735.10 Miamor Feine Filets 6 x 100 g - Huhn ...,Katzenfutter nass,Miamor,Miamor Feine Filets,Miamor Feine Filets,Finnern,Canned Food Cat / Dog,3,81276553
1671901,345861931,81276553,0,23-JAN-17,1742173,95004559,221,1.0,67798,48682,...,518433.2 Megapack Sheba Fresh & Fine Frischebe...,518433.2 Megapack Sheba Fresh & Fine Frischebe...,Katzenfutter nass,Sheba,Fresh & Fine,Fresh & Fine,Mars Petcare (Retail),Canned Food Cat / Dog,3,81276553


In [39]:
r_df , _ = rec_items(221, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)
r_df

,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION
0,3187,3187.13 Cat's Best Original Katzenstreu - 40 l...
1,276087,276087.4 Miamor Feine Filets Naturelle 6 x 80 ...
2,576075,576075.2 Multipack Sheba Classic Soup Frischeb...
3,174338,174338.9 Megapack Gourmet Perle 24 x 85 g - Er...
4,11429,11429.4 Almo Nature Jelly Pouch 6 x 70 g - Thu...
5,431494,431494.0 Almo Nature Daily Menu 6 x 85 g - Hä...
6,7786,7786.7 Miamor Feine Filets 6 x 100 g - Thunfis...
7,145745,145745.1 Almo Nature Daily Menu Pouch 6 x 70 g...
8,370544,370544.12 6 x 70 g Almo Nature Probierpaket - ...
9,514134,514134.4 Schesir Jelly Pouch 6 x 100 g - Thunf...


# Let us see how it all works!
## Add visuals to recommendations

In [40]:
##
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In order to visualize the items that were consumed by the user and visualize the corresponding recommendations I have exported additional data form the SNAP DB and reversed the function that generates direct links to the imeages

In [41]:
# load the exported data
img_df = pd.read_csv('ps_id_to_img.csv', encoding = "utf-8")#, nrows = 100000)

In [42]:
# define a lambda function that constructs the url from the image name
name_to_url_lambda = lambda x: 'https://media.zooplus.com/bilder/' + x[x.find('.')-1] + '/140/'+ x

img_df['img_url'] = img_df['IMG_NAME'].apply(name_to_url_lambda)

In [151]:
# use the model that we have traine above to generate recomendations
recommendations_df , recommended_art_c_prod_numbers = rec_items(221, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)
recommended_art_c_prod_numbers

[3187, 276087, 576075, 174338, 11429, 431494, 7786, 145745, 370544, 514134]

# ----- This seciton uses factorization obtained with the deprecated API-----
## Dsiplay the items that User has bougth

In [56]:
## DEFINE USER_ID
user_id = 10823

## get user items
user_bought_items = raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == user_id]['ART_C_PROD_NUMBER'].unique()

## get recommendations 
recommendations_df , recommended_art_c_prod_numbers = rec_items(user_id, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 15)

In [145]:
thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'User Items'+'</font>'))

for code in user_bought_items:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
    # print(img_url)

display(HTML(images))

## Display Recos for the user

In [146]:
thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'Recommended'+'</font>'))

for code in recommended_art_c_prod_numbers:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
    #print(img_url)

display(HTML(images))

# Testing multithreaded approach
> the Implicit matrix factorization is an embarrassingly parallel task and could be parallelized to multiple CPU cores as well as to GPU. The model that is used below was trained on CPU cores without the GPU usage

## Results are presented in the following way:
1. We will start with some random selected user. Display the history of his/her purchases and display the correspoinding recommendations for this user.
2. We will select cat specific user and display the history of his/her purchases and display the correspoinding recommendations for this user.
3. We will select Dog and Cat mixed user and display the history of his/her purchases and display the correspoinding recommendations for this user.
4. We will select Dog specific user with a slight bias towards cat purchases and display the history of his/her purchases and display the correspoinding recommendations for this user.
5. We will select Dpg specific user and display the history of his/her purchases and display the correspoinding recommendations for this user.

In [141]:
## DEFINE USER_ID
#user_id = 2016
print(customer_id)
## get user items
user_bought_items = raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == customer_id]['ART_C_PROD_NUMBER'].unique()

thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'User Items'+'</font>'))

for code in user_bought_items:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

58705


## Display the recommended items

In [142]:
thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=5>'+'Recommended'+'</font>'))

for code in fast_recommended:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

> One can see that our procedure has managed to capture tha fact that user was bying items for cat and for dog! The recommendation list contains both. 

### Let us try cat specific user

In [169]:
## DEFINE USER_ID
#user_id = 2016
print(customer_id)
## get user items
user_bought_items = raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == customer_id]['ART_C_PROD_NUMBER'].unique()

thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=15>'+'User Items'+'</font>'))

for code in user_bought_items:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=15>'+'Recommended'+'</font>'))

for code in fast_recommended:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

75848


> One can see that in this case the model has captured the fact that user is a cat-ony owner and reoomends only cat related products!

### Let us try a primary dog and a bit of cat

In [204]:
alpha = 15

# changing deprecated 
# user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
#                                                           factors=20, 
#                                                           regularization = 0.1, 
#                                                          iterations = 50)
model = implicit.als.AlternatingLeastSquares(factors=30, 
                                             regularization = 0.1, 
                                             iterations=10, 
                                             calculate_training_loss=True)
model.fit(((product_train*alpha).T).astype('double'))
user_items = (product_train*alpha).tocsr()

100%|██████████| 10.0/10 [00:58<00:00,  5.07s/it, loss=0.027] 


In [209]:
user_id = 1996
## 1996 heavily dog
## 123 mixed user
## 512 mixed user
## 712 cat specific
customer_id = customers_arr[user_id]
# print(customer_id)
recommendations = model.recommend(user_id, user_items, N =30)
# recommendations
fast_recommended = []
for index in recommendations:
    code = products_arr[index[0]]
    fast_recommended.append(code)
print(fast_recommended)

[604690, 319126, 207547, 307551, 169477, 93017, 637392, 183455, 468133, 304808, 467812, 482659, 104133, 136559, 305296, 468138, 102113, 537514, 348285, 586055, 533689, 497462, 220858, 604857, 586392, 320715, 579175, 3756, 590863, 59413]


In [210]:
## DEFINE USER_ID
#user_id = 2016
print(customer_id)
## get user items
user_bought_items = raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == customer_id]['ART_C_PROD_NUMBER'].unique()

thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=15>'+'User Items'+'</font>'))

for code in user_bought_items:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=15>'+'Recommended'+'</font>'))

for code in fast_recommended:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

194397


> we can see that the model captured the fact that this is primarely dog biased buyer, that it likes snacks and that it has a bit of a cat-related products!

### Purely Dog related one

In [211]:
user_id = 1997
## 1997 dog
## 1996 heavily dog
## 123 mixed user
## 512 mixed user
## 712 cat specific
customer_id = customers_arr[user_id]
# print(customer_id)
recommendations = model.recommend(user_id, user_items, N =30)
# recommendations
fast_recommended = []
for index in recommendations:
    code = products_arr[index[0]]
    fast_recommended.append(code)
print(fast_recommended)

## DEFINE USER_ID
#user_id = 2016
print(customer_id)
## get user items
user_bought_items = raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == customer_id]['ART_C_PROD_NUMBER'].unique()

thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=15>'+'User Items'+'</font>'))

for code in user_bought_items:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

thumb_html = "<img style='width: 160px; margin: 0px; \
              float: left; border: 1px solid black;' src='{}' />"
images = ''
display(HTML('<font size=15>'+'Recommended'+'</font>'))

for code in fast_recommended:
    try:
        img_url = img_df[img_df['PS_SHOP_IDENTIFIER'] == code]['img_url'].iloc[0]
    except:
        img_url = ''
    images += thumb_html.format(img_url)
   # print(img_url)

display(HTML(images))

[604690, 199688, 525558, 97648, 525680, 106179, 568382, 199684, 443556, 199736, 9650, 284568, 586674, 464268, 10344, 136547, 468133, 556050, 467686, 160389, 172956, 101516, 405292, 348285, 467812, 102952, 190513, 173310, 306751, 136559]
194453


> The model captured the fact that is a dog one :)

# Future work

1. ### Parameter Opitimizion of the latent factor model:
    1. Standard approach:
        a. Run a hyperparameter swipe/grid search (simply requires a couple of day of my CPU time. Easily doable)

1. ### Scaling
    1. Scalability to massive datasets (not a super important due to relatively 'normal' amounts of data per shop)
        * Potential problem: build a scalable recommender system that is capable of iterative update as new data is being recieved
        * Potential solution: 
            1. Due to its scalability, low memory overhead and extensibility, matrix factorization with stochastic gradient descent is our of choice (this is already the case).
            2. Paper by Rendle and Schmidt-Thieme experimentally shows that updating the latent factor vectors for only the user and item that the new rating is for, does almost as well as retraining the entire model (they report an error of at most 1%) from scratch. But they show this for explicit data, where a new rating from an exist user-item pair does mean that it over writes any previous data that we may have had for the same. This is not the case for implicit data, where, if a user buys an item a second time, we should not be ignoring the first purchase/view/like, but adding to it. - TODO

3. ### Incorporating Context features


4. ### Going wide and deep (Google paper)
[Ho Wide & Deep Learning works](https://arxiv.org/abs/1606.07792)

<p align="center">
    <img src="https://www.tensorflow.org/images/wide_n_deep.svg"/>
</p>

5. ### Implementing autoencoder for recommender system (Nvidia paper)
The [model](https://arxiv.org/abs/1708.01715), developed by NVIDIA folks, is a Deep autoencoder with 6 layers with non-linear activation function SELU (scaled exponential linear units), dropout and iterative dense refeeding.

An autoencoder is a network which implements two transformations: $encode(x): R^n \Rightarrow R^d$ and $decoder(z): R^d \Rightarrow R^n$. The “goal” of autoencoder is to obtain a $d$ dimensional representation of data such that an error measure between $x$ and $f(x) = decode(encode(x))$ is minimized. In the next figure, the autocoder architecture proposed in the [paper](https://arxiv.org/abs/1708.01715) is showed. Encoder has 2 layers $e_1$ and $e_2$ and decoder has 2 layers $d_1$ and $d_2$. Dropout may be applied to coding layer $z$. In the paper, the authors show experiments with different number of layers, from 2 to 12 (see Table 2 in the original paper).

<p align="center">
    <img src="./data/AutoEncoder.png" width=350px/>
</p>

During the forward pass the model takes a user representation by his vector of ratings from the training set $x \in R^n$, where $n$ is number of items. Note that $x$ is very sparse, while the output of the decoder, $y=f(x) \in R^n$ is dense and contains the rating predictions for all items in the corpus. The loss is the root mean squared error (RMSE).

One of the key ideas of the paper is dense re-feeding. Let's consider an idealized scenario with a perfect $f$. Then $f(x)_i = x_i ,\forall i : x_i \ne 0$ and $f(x)_i$ accurately predicts all user's future ratings. This means that if a user rates a new item $k$ (thereby creating a new vector $x'$) then $f(x)_k = x'_k$ and $f(x) = f(x')$. Therefore, the authors refeed the input in the autoencoder to augment the dataset. The method consists of the following steps:

1. Given a sparse $x$, compute the forward pass to get $f(x)$ and the loss.

2. Backpropagate the loss and update the weights.

3. Treat $f(x)$ as a new example and compute $f(f(x))$

4. Compute a second backward pass.

Steps 3 and 4 can be repeated several times.

Finally, the authors explore different non-linear [activation functions](https://github.com/pytorch/pytorch/blob/master/torch/nn/functional.py). They found that on this task ELU, SELU and LRELU, which have non-zero negative parts, perform much better than SIGMOID, RELU, RELU6, and TANH.

6. ### Comparing/Ansambling various models